In [1]:
import pandas as pd
from sklearn.cross_validation import train_test_split
import numpy as np

In [2]:
dtype_dic={'name':str,'review':str,'rating':float}

In [3]:
products=pd.read_csv('amazon_baby.csv',dtype=dtype_dic)

In [4]:
products = products.fillna({'review':''})

In [5]:
def remove_punctuation(text):
    import string
    return text.translate(None, string.punctuation) 

products['review_clean'] = products['review'].apply(remove_punctuation)

In [6]:
products=products[products['rating']!=3]

In [7]:
products.shape

(166752, 4)

In [8]:
products['sentiment'] = products['rating'].apply(lambda rating : +1 if rating > 3 else -1)

In [9]:
train_data, test_data = train_test_split(products, test_size = 0.2,random_state=1)

In [10]:
test_data.shape

(33351, 5)

In [11]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(token_pattern=r'\b\w+\b')
# Use this token pattern to keep single-letter words
# First, learn vocabulary from the training data and assign columns to words
# Then convert the training data into a sparse matrix
train_matrix = vectorizer.fit_transform(train_data['review_clean'])
# Second, convert the test data into a sparse matrix, using the same word-column mapping
test_matrix = vectorizer.transform(test_data['review_clean'])

In [12]:
sample_test_data=test_data[10:13]
print sample_test_data

                                                     name  \
117165       Lassig Glam Small Messenger Diaper Bag ,navy   
30667   BOB Weather Shield for Single Revolution/Strol...   
60268                Tiny Love Sweet Island Dreams Mobile   

                                                   review  rating  \
117165  While I'm sure this bag is a wonderful diaper ...       5   
30667   This weather shield has been a great accessory...       5   
60268   And we managed to get it to attach to the Grac...       5   

                                             review_clean  sentiment  
117165  While Im sure this bag is a wonderful diaper b...          1  
30667   This weather shield has been a great accessory...          1  
60268   And we managed to get it to attach to the Grac...          1  


In [13]:
sample_test_matrix = vectorizer.transform(sample_test_data['review_clean'])

In [14]:
from sklearn import linear_model, datasets
sentiment_model=linear_model.LogisticRegression(C=1e5)

In [15]:
train_data.head()

,name,review,rating,review_clean,sentiment
51943,Baby Einstein Take Along Tunes,,5,,1
94328,My Brest Friend Inflatable Travel Nursing Pill...,"I loved my ""Breast Friend"" nursing pillow so m...",5,I loved my Breast Friend nursing pillow so muc...,1
130352,The Grandparent Gift Co. Growing in Faith Keep...,I bought this for my baby's baptism photos. Is...,4,I bought this for my babys baptism photos Is v...,1
181820,BABYSOFT iBib - The Ultimate Silicone Baby Bib...,Well made. Silicone is very soft but stiff eno...,5,Well made Silicone is very soft but stiff enou...,1
28831,Summer Infant Secure Surround Play Safe Play Y...,We have used this extensively for the past mon...,5,We have used this extensively for the past mon...,1


In [16]:
sentiment_model.fit(train_matrix,train_data.sentiment)

LogisticRegression(C=100000.0, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0)

In [17]:
scores = sentiment_model.decision_function(sample_test_matrix)
print scores

[  9.90833917  14.25310269   0.5722351 ]


In [21]:
sss=sentiment_model.decision_function(test_matrix)
for i in range(0,100):
    print sss[i]

3.25582559396
7.02879906361
8.40859667409
7.58812608201
11.8798850401
3.65026617163
3.01185788645
6.06972038901
4.39625162744
4.14485249538
9.90833916855
14.2531026885
0.572235099413
4.28655786528
12.2518386893
-10.7325941215
5.70076355401
4.42700867893
4.07611097886
-2.55301782855
-3.37365115449
18.6352615447
6.37073752861
2.47547201895
7.54141836327
6.26194676048
12.9220076996
3.12969594119
-0.669881290323
14.0639823547
3.74731101059
8.77519660386
12.443374445
5.87170473543
13.5845363363
12.3893051891
1.86855973639
6.3844435973
7.8948138838
4.00490193328
9.89008188695
2.96555797911
-11.217974187
8.7972064968
2.87799465219
-2.04761169179
22.3812559915
7.50488388284
-13.768811455
9.55479245809
-7.53149651418
7.95668272181
10.2774269724
8.50026483746
9.25755448557
11.102316853
6.04980047216
2.49779534683
1.04111656673
10.3509654358
4.59165771964
-16.6339903225
3.63718314214
4.52884738647
11.456240367
5.52159791258
7.58525510276
1.55753316031
8.95184645528
17.3614085304
5.86375514441
2.8

In [22]:
result=sentiment_model.predict(test_matrix)

In [23]:
result

array([1, 1, 1, ..., 1, 1, 1])

In [24]:
test_data=test_data.reset_index(drop=True)

In [25]:
new=pd.concat([test_data,pd.DataFrame(result)],axis=1)

In [26]:
(new['sentiment']==new[0]).sum()*1.0/new.shape[0]

0.92836796497856133